In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df = con.execute("""
                 SELECT * 
                 FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-01-11'
                ) WHERE row = 1
                 """).fetchdf()
df.head(10)

In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})
df_final.head(10)

In [ ]:
df_final.dtypes

In [ ]:
df2 = df_final
df2 = df2.astype(
    {
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
    }
)

df2.dtypes


In [ ]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [ ]:
df2.head(10)

In [ ]:
con.execute("INSERT INTO produtos SELECT * from df2")

In [ ]:
df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head(10)

In [32]:
con.close()